In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv("diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

No Missing Values present

In [4]:
df.duplicated().sum()

0

No duplicated value Present

## Dependent and Independent Features

In [5]:
X=df.drop(columns=['Outcome']).values
y=df['Outcome'].values
print(X.shape)
print(y.shape)


(768, 8)
(768,)


## Dataset Preparation for training

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [7]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [8]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(614, 8)
(154, 8)
(614,)
(154,)


In [9]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

## Converting numpy array to Tensors

In [98]:
X_train_tensor=torch.from_numpy(X_train_scaled.astype(np.float32))
X_test_tensor=torch.from_numpy(X_test_scaled.astype(np.float32))
y_train_tensor=torch.from_numpy(y_train.astype(np.float32))
y_test_tensor=torch.from_numpy(y_test.astype(np.float32))

In [99]:
X_train_tensor

tensor([[ 1.5166,  0.7505,  0.5648,  ...,  0.7958,  0.5295,  0.5679],
        [ 1.8120,  0.2448, -0.3479,  ...,  1.2287, -0.0697,  0.3984],
        [ 0.9257, -0.6087,  0.2605,  ...,  0.7040, -0.7942,  0.9916],
        ...,
        [ 2.6983,  0.1499,  1.0718,  ...,  1.5172,  0.3644,  0.7374],
        [ 0.0395,  1.5724,  0.1591,  ...,  1.5434,  0.0465, -0.6184],
        [ 1.5166, -0.6087,  0.3619,  ...,  0.1400,  0.6151,  1.0764]])

##  Basic Model Building using Pytorch

In [100]:
class MyNet(nn.Module):
    """Some Information about MyModule"""
    def __init__(self,input_size):
        super().__init__()
        self.first_layer=nn.Linear(input_size,1,dtype=torch.float32)
        self.sigmoid=nn.Sigmoid()

    def forward(self, features):
        out=self.first_layer(features)
        out=self.sigmoid(out)
        return out


In [101]:
model=MyNet(X_train_tensor.shape[1])
# y_pred=model(X_train_tensor)

In [102]:
loss_function = nn.BCELoss()
optimizer=torch.optim.AdamW(model.parameters(), lr=0.1)

In [103]:
for i in range(30):
    y_pred=model(X_train_tensor)
    loss=loss_function(y_pred,y_train_tensor.view(-1,1))
    print(f"epoch: {i+1} Loss: {loss}")
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    


epoch: 1 Loss: 0.7385072708129883
epoch: 2 Loss: 0.6715041995048523
epoch: 3 Loss: 0.6214901804924011
epoch: 4 Loss: 0.5843688249588013
epoch: 5 Loss: 0.5564995408058167
epoch: 6 Loss: 0.5348109006881714
epoch: 7 Loss: 0.5175073742866516
epoch: 8 Loss: 0.5040386915206909
epoch: 9 Loss: 0.494070827960968
epoch: 10 Loss: 0.48720577359199524
epoch: 11 Loss: 0.48293250799179077
epoch: 12 Loss: 0.48056039214134216
epoch: 13 Loss: 0.47925978899002075
epoch: 14 Loss: 0.47837984561920166
epoch: 15 Loss: 0.4776502549648285
epoch: 16 Loss: 0.4770512878894806
epoch: 17 Loss: 0.47662854194641113
epoch: 18 Loss: 0.47640708088874817
epoch: 19 Loss: 0.47639262676239014
epoch: 20 Loss: 0.4765886962413788
epoch: 21 Loss: 0.47698283195495605
epoch: 22 Loss: 0.47751301527023315
epoch: 23 Loss: 0.47806182503700256
epoch: 24 Loss: 0.47849714756011963
epoch: 25 Loss: 0.47872427105903625
epoch: 26 Loss: 0.4787120521068573
epoch: 27 Loss: 0.4784848988056183
epoch: 28 Loss: 0.4780963957309723
epoch: 29 Loss: 0

In [104]:
model.first_layer.weight

Parameter containing:
tensor([[ 0.4725,  1.4507, -0.2517,  0.0231, -0.2011,  0.6583,  0.2322,  0.0678]],
       requires_grad=True)

In [66]:
model.first_layer.weight

Parameter containing:
tensor([[ 0.2397,  0.1899, -0.0900,  0.2251,  0.1139,  0.1430, -0.0517,  0.2978]],
       dtype=torch.float64, requires_grad=True)

## Multilayer Model using Pytorch

In [246]:
class MyMLModel(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.network=nn.Sequential(
            nn.Linear(num_features,15),
            nn.ReLU(),
            nn.Linear(15,7),
            nn.ReLU(),
            nn.Linear(7,3),
            nn.ReLU(),
            nn.Linear(3,1),
            nn.Sigmoid()

        )
    def forward(self,features):
        out=self.network(features)
        return out

In [276]:
newmodel=MyMLModel(X_train_tensor.shape[1])

In [277]:
loss_function = nn.BCELoss()
optimizer=torch.optim.AdamW(newmodel.parameters(), lr=0.1)

In [262]:
optimizer.param_groups[0]['lr']=0.3

In [263]:
optimizer.param_groups[0]['lr']

0.3

In [278]:
for i in range(200):
    # Training
    y_pred = newmodel(X_train_tensor)
    loss = loss_function(y_pred, y_train_tensor.view(-1, 1))
    
    # Validation
    with torch.no_grad():
        y_pred_val = newmodel(X_test_tensor)
        val_loss = loss_function(y_pred_val, y_test_tensor.view(-1, 1))
        y_pred_val_binary = (y_pred_val > 0.3).float()
        val_accuracy = (y_pred_val_binary == y_test_tensor.view(-1, 1)).sum() / len(y_test_tensor)
    
    print(f"Epoch: {i+1}, Loss: {loss}, Val Loss: {val_loss}, Val Accuracy: {val_accuracy:.4f}")
    
    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    optimizer.param_groups[0]['lr']=loss/10

Epoch: 1, Loss: 0.6710340976715088, Val Loss: 0.6723341345787048, Val Accuracy: 0.3571
Epoch: 2, Loss: 0.6638672947883606, Val Loss: 0.6681098341941833, Val Accuracy: 0.3571
Epoch: 3, Loss: 0.6544120907783508, Val Loss: 0.6580721139907837, Val Accuracy: 0.3571
Epoch: 4, Loss: 0.6507963538169861, Val Loss: 0.6550838947296143, Val Accuracy: 0.3571
Epoch: 5, Loss: 0.6482145190238953, Val Loss: 0.6531212329864502, Val Accuracy: 0.3571
Epoch: 6, Loss: 0.6465647220611572, Val Loss: 0.6520583629608154, Val Accuracy: 0.3571
Epoch: 7, Loss: 0.6457180380821228, Val Loss: 0.6517570614814758, Val Accuracy: 0.3571
Epoch: 8, Loss: 0.6455132961273193, Val Loss: 0.6520459055900574, Val Accuracy: 0.3571
Epoch: 9, Loss: 0.6457650661468506, Val Loss: 0.6527288556098938, Val Accuracy: 0.3571
Epoch: 10, Loss: 0.6462785005569458, Val Loss: 0.6536023020744324, Val Accuracy: 0.3571
Epoch: 11, Loss: 0.6468722224235535, Val Loss: 0.6544783115386963, Val Accuracy: 0.3571
Epoch: 12, Loss: 0.6474003195762634, Val 

In [194]:
from sklearn.metrics import accuracy_score

In [215]:
y_test_pred=newmodel(X_test_tensor)

In [216]:
y_pred_binary = (y_test_pred > 0.5).float()

In [217]:
y_pred_binary=torch.tensor(y_pred_binary,dtype=torch.float32)

C:\Users\djdeb\AppData\Local\Temp\ipykernel_5776\3705519333.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred_binary=torch.tensor(y_pred_binary,dtype=torch.float32)


In [218]:
accuracy_score(y_pred_binary,y_test_tensor)

0.7467532467532467